In [ ]:
#gameId = "doapp"
gameId = "sfiii3n"
#gameId = "umk3"
#gameId = "tektagt"

In [ ]:
from collections import namedtuple
# Algorithm options
algoOpt = namedtuple('algoOptions', 'character player frameSize frameStack actionSpace attackButComb\
                                     frameRatio actionBufLen dilation')
if gameId != "tektagt":
    algoOpt.character = "Random"
elif gameId == "tektagt":
    algoOpt.character = ["Random", "Random"]

algoOpt.frameRatio    = 6
algoOpt.player        = "Random"
algoOpt.frameSize     = 128
algoOpt.frameStack    = 4
algoOpt.dilation      = 1
algoOpt.actionSpace   = "discrete"
algoOpt.attackButComb = True
algoOpt.actionBufLen  = 12

In [ ]:
import sys, os, time
import numpy as np

timeDepSeed = int((time.time()-int(time.time()-0.5))*1000)

sys.path.append(os.path.join(os.path.abspath(''), '../'))  
sys.path.append(os.path.join(os.path.abspath(''), '../../gym/'))
                                                                                                                                 
from gymUtils import discreteToMultiDiscreteAction
from sbUtils import showObs
from makeStableBaselinesEnv import makeStableBaselinesEnv

In [ ]:
# Common settings
diambraKwargs = {}
diambraKwargs["romsPath"] = "../../roms/mame/"
diambraKwargs["binaryPath"] = "../../customMAME/"
diambraKwargs["frameRatio"] = algoOpt.frameRatio
diambraKwargs["throttle"] = False
diambraKwargs["sound"] = diambraKwargs["throttle"]

diambraKwargs["player"] = algoOpt.player

diambraKwargs["characters"] =[algoOpt.character, algoOpt.character]
diambraKwargs["charOutfits"] = [2, 2]

In [ ]:
# DIAMBRA gym kwargs
diambraGymKwargs = {}
diambraGymKwargs["actionSpace"] = [algoOpt.actionSpace, algoOpt.actionSpace]
diambraGymKwargs["attackButCombinations"] = [algoOpt.attackButComb, algoOpt.attackButComb]
diambraGymKwargs["actBufLen"] = algoOpt.actionBufLen
if diambraKwargs["player"] != "P1P2":
    diambraGymKwargs["showFinal"] = False
    diambraGymKwargs["continueGame"] = -1.0
    diambraGymKwargs["actionSpace"] = diambraGymKwargs["actionSpace"][0]
    diambraGymKwargs["attackButCombinations"] = diambraGymKwargs["attackButCombinations"][0]
    
idxList = [0, 1]
if diambraKwargs["player"] != "P1P2":
    idxList = [0]
    
# Env wrappers kwargs
wrapperKwargs = {}
wrapperKwargs["noOpMax"] = 0
wrapperKwargs["hwcObsResize"] = [algoOpt.frameSize, algoOpt.frameSize, 1]
wrapperKwargs["normalizeRewards"] = True
wrapperKwargs["clipRewards"] = False
wrapperKwargs["frameStack"] = algoOpt.frameStack
wrapperKwargs["dilation"] = algoOpt.dilation
wrapperKwargs["scale"] = True
wrapperKwargs["scaleMod"] = 0

# Additional obs key list
keyToAdd = []
keyToAdd.append("actionsBuf")

if gameId != "tektagt":                                                         
    keyToAdd.append("ownHealth")
    keyToAdd.append("oppHealth")
    if gameId == "sfiii3n":
        keyToAdd.append("ownSuperBarVal")
        keyToAdd.append("ownSuperCount")
        keyToAdd.append("ownSuperMaxCount")
        keyToAdd.append("ownSuperSelected")
        keyToAdd.append("ownStunBar")
        keyToAdd.append("ownStunned")
        keyToAdd.append("oppSuperBarVal")
        keyToAdd.append("oppSuperCount")
        keyToAdd.append("oppSuperMaxCount")
        keyToAdd.append("oppSuperSelected")
        keyToAdd.append("oppStunBar")
        keyToAdd.append("oppStunned")
else:                                                                           
    keyToAdd.append("ownHealth1")
    keyToAdd.append("ownHealth2")
    keyToAdd.append("oppHealth1")
    keyToAdd.append("oppHealth2")
    keyToAdd.append("ownActiveChar")
    keyToAdd.append("oppActiveChar")
    
keyToAdd.append("ownPosition")
keyToAdd.append("oppPosition")
if diambraKwargs["player"] != "P1P2":
    keyToAdd.append("stage")
keyToAdd.append("ownChar")
keyToAdd.append("oppChar")

In [ ]:
hardCore = False
envId = gameId + "_randomTestSBWrap"
numOfEnvs = 1
env = makeStableBaselinesEnv(envId, numOfEnvs, timeDepSeed, diambraKwargs, diambraGymKwargs, 
                             wrapperKwargs, keyToAdd=keyToAdd, p2Mode=None, 
                             p2Policy=None,noVec=True, hardCore=hardCore)

In [ ]:
print("Observation Space:", env.observation_space)
print("Action Space:", env.action_space)
if not hardCore:
    print("Keys to Dict:")
    for k,v in env.keysToDict.items():
        print(k, v)

In [ ]:
nActions = env.nActions
if diambraKwargs["player"] != "P1P2":
    nActions=[env.nActions]

In [ ]:
actionsPrintDict = env.printActionsDict()
observation = env.reset()

In [ ]:
showObs(observation, keyToAdd, env.keyToAddCount, env.actBufLen, nActions, 
        0, True, env.charNames, hardCore, idxList)

In [ ]:
cumulativeEpRew = 0.0
cumulativeEpRewAll = []

maxNumEp = 100
currNumEp = 0

while currNumEp < maxNumEp:

    actions = [None, None]
    if diambraKwargs["player"] != "P1P2":
        actions = env.action_space.sample()
        
        if diambraGymKwargs["actionSpace"] == "discrete":
            moveAction, attAction = discreteToMultiDiscreteAction(actions, env.nActions[0])
        else:
            moveAction, attAction = actions[0], actions[1]
            
        print("(P1) {} {}".format(actionsPrintDict[0][moveAction],       
                                  actionsPrintDict[1][attAction])) 
        
    else:
        
        for idx in range(2):
        
            actions[idx] = env.action_space["P{}".format(idx+1)].sample()

            if diambraGymKwargs["actionSpace"][idx] == "discrete":
                moveAction, attAction = discreteToMultiDiscreteAction(actions[idx], env.nActions[idx][0])
            else:
                moveAction, attAction = actions[idx][0], actions[idx][1]
        
            if diambraKwargs["player"] != "P1P2" and idx == 1:
                continue
            
            print("(P{}) {} {}".format(idx+1, actionsPrintDict[0][moveAction],       
                                              actionsPrintDict[1][attAction])) 
        
    if diambraKwargs["player"] == "P1P2" or diambraGymKwargs["actionSpace"] != "discrete":
        actions = np.append(actions[0], actions[1])   
    
    observation, reward, done, info = env.step(actions)
    
    print("action = ", actions)
    print("reward:", reward)
    print("done = ", done)
    for k, v in info.items():
        print("info[\"{}\"] = {}".format(k, v))
    showObs(observation, keyToAdd, env.keyToAddCount, env.actBufLen, nActions, 
            0, True, env.charNames, hardCore, idxList)
        
    print("----------")
    
    cumulativeEpRew += reward
    
    if np.any(done):
        currNumEp += 1
        print("Ep. # = ", currNumEp)
        print("Ep. Cumulative Rew # = ", cumulativeEpRew)
        cumulativeEpRewAll.append(cumulativeEpRew)
        cumulativeEpRew = 0.0

        observation = env.reset()
        showObs(observation, keyToAdd, env.keyToAddCount, env.actBufLen, nActions, 
                0, True, env.charNames, hardCore, idxList)
        
print("Mean cumulative reward = ", np.mean(cumulativeEpRewAll))    
print("Std cumulative reward = ", np.std(cumulativeEpRewAll))       
    
env.close()